In [ ]:
import pandas as pd
import os
import sys
import time
import pandas as pd
import requests


In [ ]:
from datetime import datetime
import pytz

from joblib.logger import PrintTime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor

import numpy as np
from joblib.logger import PrintTime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

file_path = 'drive/MyDrive/data/'
resultsDict = {}

for filename in os.listdir(file_path):
  if filename.endswith(".csv"):
    energyData = pd.read_csv(file_path + filename)

#get data
    res = filename.split("_", 1)
    airport = res[1][:4]
    
    energyData['Dates'] = pd.to_datetime(energyData['Date/Time']).dt.date
    energyData['Time'] = pd.to_datetime(energyData['Date/Time']).dt.time

    energyData['Dates'] = energyData['Dates'].astype(str)
    energyData["Year"] = energyData['Dates'].str.slice(0,4)
    energyData["Month"] = energyData['Dates'].str.slice(5,7)
    energyData["Day"] = energyData['Dates'].str.slice(8,10)
    energyData["DateAPIFormat"] = energyData.Year + energyData.Month + energyData.Day

    energyData
    energyData
    

    # get sunset/sunrise data: #FIX API KEY
    HB_PAN = [35.371452, -101.387665]
    date = 20230318
    # sunrise = requests.get("https://api.sunrise-sunset.org/json?lat=35.371452&lng=-101.3887665&date=20230318")

    sunriseinfo = {}
    sunsetinfo = {}
    daylengthinfo = {}
    #takes 2.5mins to run
    for day in energyData["DateAPIFormat"].unique():
        # sunrise = requests.get("https://api.sunrise-sunset.org/json?lat=34.963623&lng=-120.424576&date=%s&formatted=0"%day)
        res = sunrise.json()["results"]
        sunriseinfo[day] =res["sunrise"]
        sunsetinfo[day] =res["sunset"]
        daylengthinfo[day]=res["day_length"]

        sys.stdout.write('.')
        time.sleep(.0001)
    
    pd.to_datetime(sunsetinfo[energyData["DateAPIFormat"].unique()[0]])

    energyData["Sunrise"] = energyData["DateAPIFormat"].map(sunriseinfo)
    energyData["Sunset"] = energyData["DateAPIFormat"].map(sunsetinfo)
    energyData["Day_Length"] = energyData["DateAPIFormat"].map(daylengthinfo)
    energyData

    weather16_17 = requests.get('https://api.synopticlabs.org/v2/stations/timeseries?&token=3066f4e105ff4614a61897b282e96cae&stid='+airport+'&start=202203180000&end=202303182000&timeformat=Y%25YM%25mD%25dH%25HM%25M&vars=air_temp,weather_condition,relative_humidity,wind_speed,wind_direction,wind_gust,solar_radiation,peak_wind_speed,pressure,cloud_layer_1_code')
    weatherData = pd.DataFrame(weather16_17.json()['STATION'][0]["OBSERVATIONS"])
    # weather_df.to_csv("Weather.csv", index=False)
    energyData.to_csv("Energy.csv")
    # weatherData = pd.read_csv("Weather.csv")
    time.sleep(.001)
    energyData2 = pd.read_csv("Energy.csv")


    #clean data to local timezone!
    local = pytz.timezone("US/Pacific")
    weatherData.date_time = pd.to_datetime(weatherData.date_time, format='Y%YM%mD%dH%HM%M')
    weatherData.date_time = weatherData.date_time.dt.tz_localize("UTC").dt.tz_convert(local)
    print(energyData2)
    energyData2.Sunrise = pd.to_datetime(energyData2.Sunrise, format='%Y-%m-%dT%H:%M:%S')
    energyData2.Sunset = pd.to_datetime(energyData2.Sunset, format='%Y-%m-%dT%H:%M:%S')
    energyData2.Sunrise = energyData2.Sunrise.dt.tz_convert(local)
    energyData2.Sunset = energyData2.Sunset.dt.tz_convert(local)
    pd.set_option('display.max_columns', None)
    


    #convert date_time to same style as other one
    weatherData["dateJoinFormat"] = weatherData["date_time"].dt.strftime("%m/%d/%Y %H:%M:%S").str[:19]
    weatherData


    energyData2["Date/Time"]=pd.to_datetime(energyData2['Date/Time'])
    energyData2["dateJoinFormat"]=energyData2["Date/Time"].dt.strftime("%m/%d/%Y %H:%M:%S")
    weatherData.drop_duplicates(subset="dateJoinFormat",
                      keep=False, inplace=True)
    energywithweather = pd.merge(energyData2,weatherData,how='inner',on='dateJoinFormat',validate='one_to_one')

    energyData2["Date/Time"]=pd.to_datetime(energyData2['Date/Time'])
    energyData2["dateJoinFormat"]=energyData2["Date/Time"].dt.strftime("%m/%d/%Y %H:%M:%S")
    weatherData.drop_duplicates(subset="dateJoinFormat",
                      keep=False, inplace=True)
    energywithweather = pd.merge(energyData2,weatherData,how='inner',on='dateJoinFormat',validate='one_to_one')


    ##DATA ANALYSIS
    data = energywithweather
    dataclean = data.iloc[10:,:]
    dataclean.to_csv("dataclean.csv")

    #MACHINE LEARNING
    data['weather_condition_set_1d' ] = data['weather_condition_set_1d'].fillna('')
    data['cloud_layer_1_code' ] = data['cloud_layer_1_code'].astype(int)
    data['cloud_layer_1_code'] = data['cloud_layer_1_code'].str.strip().str[-1]
    data['cloud_layer_1_code' ] = data['cloud_layer_1_code'].fillna('')


    data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
    data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
    data['Return %'] = data['Return %'].str.replace('%', '').astype(float)/100
    data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace('$', '').astype(float)
    data['Load '] = data['Load '].str.replace(',', '').str.replace('$', '').astype(float)





    X_train, X_test, y_train, y_test = train_test_split(data[["Day","Hour", "Gas Price $/mmBtu", "Load ", "Day_Length", "Temperature °F",'weather_condition_set_1d',
          'pressure_set_1d', 'relative_humidity_set_1', 'Wind Speed MPH', 'air_temp_set_1' ]],
                                                        data["Return $/MW"], 
                                                        test_size=0.1, #using 10% to train for section speedup
                                                        random_state=710)
    #Model Training Code

    # model = KNeighborsClassifier(10)
    model = LGBMRegressor()
    pipeline = make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["weather_condition_set_1d"]),
            remainder="passthrough"),
            StandardScaler(),
        model)


    resultsDict[filename.split('_')[0]+"mse"] = (-cross_val_score(
        pipeline,
        X_train, y_train,
        scoring="neg_mean_squared_error",
        cv=10).mean())



        #PREDICT BUY SELL
    data["direction"] = np.sign(data["Return $/MW"])

    le.fit(data['direction'])
    data['direction'] = le.fit_transform(data['direction'])

    X_train, X_test, y_train, y_test = train_test_split(data[["Day","Hour", "Gas Price $/mmBtu", "Load ", "Day_Length", "Temperature °F",'weather_condition_set_1d',
          'pressure_set_1d', 'relative_humidity_set_1', 'Wind Speed MPH', 'air_temp_set_1' ]],
                                                        data["direction"], 
                                                        test_size=0.1, #using 10% to train for section speedup
                                                        random_state=710)
    #Model Training Code

    # model = KNeighborsClassifier(10)
    model = LGBMClassifier()
    pipeline = make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["weather_condition_set_1d"]),
            remainder="passthrough"),
            StandardScaler(),
        model)

    resultsDict[filename+"accuracy"] = cross_val_score(
        pipeline,
        X_train, y_train,
        scoring="accuracy",
        cv=10).mean()

    pipeline = make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["weather_condition_set_1d"]),
            remainder="passthrough"),
            StandardScaler(),
        model)

    resultsDict[filename+"f1"] = cross_val_score(
    pipeline,
    X_train, y_train,
    scoring="f1_macro",
    cv=10).mean()

print(resultsDict)


...............................................................................................................................................................................................................................................................................................................................................................................      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  4-26-2022 11:00   26  11.0       -$37.85        $42.19   
1              1   2-6-2023 03:00    6   3.0       -$31.14         $0.00   
2              2   2-6-2023 01:00    6   1.0       -$31.06        -$0.56   
3              3   2-6-2023 02:00    6   2.0       -$30.99         $0.35   
4              4   2-6-2023 04:00    6   4.0       -$30.97         $0.03   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,629.86       $639.19   
8779        8779  5-13-20

<ipython-input-19-5afca7d39a96>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:128: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

...............................................................................................................................................................................................................................................................................................................................................................................      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  11-4-2022 20:00    4  20.0      -$173.18        $51.11   
1              1   2-6-2023 03:00    6   3.0       -$31.56         $3.11   
2              2   2-6-2023 04:00    6   4.0       -$31.54         $3.62   
3              3   2-6-2023 01:00    6   1.0       -$31.50         $3.05   
4              4   2-6-2023 02:00    6   2.0       -$31.49         $3.16   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,622.06       $642.13   
8779        8779  5-13-20

<ipython-input-19-5afca7d39a96>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:128: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

...............................................................................................................................................................................................................................................................................................................................................................................      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  11-4-2022 20:00    4  20.0      -$228.74        $53.60   
1              1   2-6-2023 03:00    6   3.0       -$31.63         $3.49   
2              2   2-6-2023 04:00    6   4.0       -$31.62         $4.06   
3              3   2-6-2023 02:00    6   2.0       -$31.60         $3.52   
4              4   2-6-2023 01:00    6   1.0       -$31.60         $3.49   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,628.67       $643.29   
8779        8779  5-13-20

...

...............................................................................................................................................................................................................................................................................................................................................................................      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0   2-6-2023 03:00    6   3.0       -$31.58         $3.80   
1              1   2-6-2023 04:00    6   4.0       -$31.56         $4.43   
2              2   2-6-2023 01:00    6   1.0       -$31.52         $3.85   
3              3   2-6-2023 02:00    6   2.0       -$31.52         $3.86   
4              4   1-2-2023 04:00    2   4.0       -$30.36         $3.71   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,619.14       $643.09   
8779        8779  5-13-20

<ipython-input-19-5afca7d39a96>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:128: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

...............................................................................................................................................................................................................................................................................................................................................................................      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  3-18-2022 05:00   18   5.0       -$49.96        -$4.15   
1              1  3-18-2022 04:00   18   4.0       -$35.69        -$5.35   
2              2  4-17-2022 21:00   17  21.0       -$33.07        $82.76   
3              3   2-6-2023 02:00    6   2.0       -$32.11        -$2.33   
4              4   2-6-2023 01:00    6   1.0       -$32.08        -$3.20   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,629.79       $672.22   
8779        8779  5-13-20

<ipython-input-19-5afca7d39a96>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:128: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-19-5afca7d39a96>:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

{'HBWestmse': 14836.35777849786, 'HBWest_KSJT.csvaccuracy': 0.7509766368521233, 'HBWest_KSJT.csvf1': 0.5328602714010432, 'HBSouthmse': 13323.953538538883, 'HBSouth_KSJT.csvaccuracy': 0.7787562113632153, 'HBSouth_KSJT.csvf1': 0.6877353007090496, 'HBNorthmse': 15616.051886420599, 'HBNorth_KDAL.csvaccuracy': 0.7591916558018253, 'HBNorth_KDAL.csvf1': 0.6440974768291199, 'TOPAZALLmse': 20158.446320297542, 'TOPAZALL_KGLS.csvaccuracy': 0.736665973233479, 'TOPAZALL_KGLS.csvf1': 0.5640283683094981, 'HBHoustonmse': 21014.17190593366, 'HBHouston_KIAH.csvaccuracy': 0.7878228645543821, 'HBHouston_KIAH.csvf1': 0.667354356367968, 'BRAZWINDmse': 15019.091311257982, 'BRAZWIND_KLBB.csvaccuracy': 0.7896466897693529, 'BRAZWIND_KLBB.csvf1': 0.6903244092504743}


In [ ]:
print(resultsDict)

In [ ]:
energyData

In [ ]:
weatherData

,date_time,wind_gust_set_1,weather_condition_set_1d,pressure_set_1d,cloud_layer_3_code_set_1,wind_direction_set_1,wind_speed_set_1,relative_humidity_set_1,peak_wind_speed_set_1,air_temp_set_1,dateJoinFormat
0,2022-03-17 17:00:00-07:00,22.121,blowing dust,89713.91,NaN,340.0,15.433,49.51,NaN,9.0,03/17/2022 17:00:00
1,2022-03-17 17:05:00-07:00,19.549,blowing dust,89713.91,NaN,350.0,15.433,52.98,NaN,8.0,03/17/2022 17:05:00
2,2022-03-17 17:10:00-07:00,21.092,blowing dust,89743.95,NaN,340.0,14.404,52.98,NaN,8.0,03/17/2022 17:10:00
3,2022-03-17 17:15:00-07:00,22.121,blowing dust,89773.98,NaN,340.0,16.977,52.98,NaN,8.0,03/17/2022 17:15:00
4,2022-03-17 17:20:00-07:00,20.063,blowing dust,89773.98,NaN,350.0,15.948,52.98,NaN,8.0,03/17/2022 17:20:00
...,...,...,...,...,...,...,...,...,...,...,...
112701,2023-03-18 12:45:00-07:00,NaN,None,91185.62,NaN,70.0,3.601,23.04,NaN,9.0,03/18/2023 12:45:00
112702,2023-03-18 12:50:00-07:00,7.202,None,91155.58,NaN,100.0,4.630,23.04,NaN,9.0,03/18/2023 12:50:00
112703,2023-03-18 12:53:00-07:00,7.717,None,91155.58,NaN,NaN,2.572,23.01,NaN,8.9,03/18/2023 12:53:00
112704,2023-03-18 12:55:00-07:00,NaN,None,91155.58,NaN,70.0,1.543,22.75,NaN,8.0,03/18/2023 12:55:00


In [ ]:
weather16_17

<Response [200]>

In [ ]:
energywithweather

,Unnamed: 0,Date/Time,Day,Hour,RT Price $/MW,DA Price $/MW,Gas Price $/mmBtu,Gas Basis $/mmBtu,Load,Temperature °F,Wind Speed MPH,RT Heat Rate,DA Heat Rate,Return $,Return $/MW,Return %,Cum. Return $,Dates,Time,Year,Month,DateAPIFormat,Sunrise,Sunset,Day_Length,dateJoinFormat,date_time,wind_gust_set_1,weather_condition_set_1d,pressure_set_1d,wind_direction_set_1,wind_speed_set_1,relative_humidity_set_1,peak_wind_speed_set_1,air_temp_set_1
0,0,2022-03-18 05:00:00,18,5.0,-$49.96,-$4.15,$4.80,$0.00,"5,554.63",37.0,15.0,-10.41,-0.86,-$45.81,-$45.81,1103.80%,-$68.04,2022-03-18,05:00:00,2022,3,20220318,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,03/18/2022 05:00:00,2022-03-18 05:00:00-07:00,NaN,None,90464.78,300.0,5.144,74.41,NaN,0.0
1,1,2022-03-18 04:00:00,18,4.0,-$35.69,-$5.35,$4.80,$0.00,"5,479.75",37.9,16.1,-7.43,-1.11,-$30.34,-$30.34,567.01%,-$22.23,2022-03-18,04:00:00,2022,3,20220318,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,03/18/2022 04:00:00,2022-03-18 04:00:00-07:00,NaN,None,90434.75,310.0,6.173,74.60,NaN,1.0
2,2,2022-04-17 21:00:00,17,21.0,-$33.07,$82.76,$6.93,$0.00,"6,086.86",70.0,9.2,-4.77,11.94,-$115.83,-$115.83,-139.96%,-$625.19,2022-04-17,21:00:00,2022,4,20220417,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,04/17/2022 21:00:00,2022-04-17 21:00:00-07:00,NaN,None,90254.54,120.0,5.144,25.04,NaN,16.0
3,3,2023-02-06 02:00:00,6,2.0,-$32.11,-$2.33,$2.40,$0.00,"6,336.77",54.0,19.6,-13.38,-0.97,-$29.78,-$29.78,1278.11%,"-$34,169.46",2023-02-06,02:00:00,2023,2,20230206,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,02/06/2023 02:00:00,2023-02-06 02:00:00-08:00,12.347,None,89743.95,220.0,9.774,53.53,NaN,10.0
4,4,2023-02-06 01:00:00,6,1.0,-$32.08,-$3.20,$2.40,$0.00,"6,342.15",55.0,19.6,-13.37,-1.33,-$28.88,-$28.88,902.50%,"-$34,139.68",2023-02-06,01:00:00,2023,2,20230206,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,02/06/2023 01:00:00,2023-02-06 01:00:00-08:00,NaN,None,89804.02,220.0,7.202,50.08,NaN,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8579,8778,2022-07-13 15:00:00,13,15.0,"$3,629.79",$672.22,$6.79,$0.00,"7,377.88",96.1,11.5,534.58,99.00,"$2,957.57","$2,957.57",439.97%,"-$14,382.14",2022-07-13,15:00:00,2022,7,20220713,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,07/13/2022 15:00:00,2022-07-13 15:00:00-07:00,NaN,None,90374.68,120.0,4.116,16.01,NaN,37.0
8580,8779,2022-05-13 17:00:00,13,17.0,"$4,511.76",$600.37,$7.31,$0.00,"6,959.68",91.9,0.0,617.20,82.13,"$3,911.39","$3,911.39",651.50%,-$29.39,2022-05-13,17:00:00,2022,5,20220513,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,05/13/2022 17:00:00,2022-05-13 17:00:00-07:00,NaN,None,89864.09,90.0,3.087,4.86,NaN,33.0
8581,8780,2022-07-13 16:00:00,13,16.0,"$5,094.77",$921.58,$6.79,$0.00,"7,365.63",97.0,8.1,750.33,135.73,"$4,173.19","$4,173.19",452.83%,"-$10,208.95",2022-07-13,16:00:00,2022,7,20220713,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,07/13/2022 16:00:00,2022-07-13 16:00:00-07:00,NaN,None,90314.61,120.0,3.087,15.78,NaN,36.0
8582,8781,2022-07-13 17:00:00,13,17.0,"$5,278.86",$780.64,$6.79,$0.00,"7,371.97",98.1,9.2,777.45,114.97,"$4,498.22","$4,498.22",576.22%,"-$5,710.73",2022-07-13,17:00:00,2022,7,20220713,2022-08-24 06:27:32-07:00,2022-08-24 19:40:25-07:00,47573,07/13/2022 17:00:00,2022-07-13 17:00:00-07:00,NaN,None,90284.57,120.0,5.659,15.78,NaN,36.0


In [ ]:
    weather16_17 = requests.get('https://api.synopticlabs.org/v2/stations/timeseries?&token=3066f4e105ff4614a61897b282e96cae&stid='+airport+'&start=202203180000&end=202303182000&timeformat=Y%25YM%25mD%25dH%25HM%25M&vars=air_temp,weather_condition,relative_humidity,wind_speed,wind_direction,wind_gust,solar_radiation,peak_wind_speed,pressure,cloud_layer_1_code, incoming_radiation_uv, net_radiation, visibility,visibility_code')


In [ ]:
    weatherData = pd.DataFrame(weather16_17.json()['STATION'][0]["OBSERVATIONS"])
    pd.set_option('display.max_rows', 500)
    weatherData.to_csv("csv.csv")

In [ ]:
weatherData

,date_time,wind_gust_set_1,weather_condition_set_1d,pressure_set_1d,cloud_layer_1_code_set_1,wind_speed_set_1,relative_humidity_set_1,peak_wind_speed_set_1,air_temp_set_1,wind_direction_set_1
0,Y2022M03D18H00M00,22.121,blowing dust,89713.91,1.0,15.433,49.51,NaN,9.0,340.0
1,Y2022M03D18H00M05,19.549,blowing dust,89713.91,1.0,15.433,52.98,NaN,8.0,350.0
2,Y2022M03D18H00M10,21.092,blowing dust,89743.95,1.0,14.404,52.98,NaN,8.0,340.0
3,Y2022M03D18H00M15,22.121,blowing dust,89773.98,1.0,16.977,52.98,NaN,8.0,340.0
4,Y2022M03D18H00M20,20.063,blowing dust,89773.98,1.0,15.948,52.98,NaN,8.0,350.0
...,...,...,...,...,...,...,...,...,...,...
112701,Y2023M03D18H19M45,NaN,None,91185.62,706.0,3.601,23.04,NaN,9.0,70.0
112702,Y2023M03D18H19M50,7.202,None,91155.58,706.0,4.630,23.04,NaN,9.0,100.0
112703,Y2023M03D18H19M53,7.717,None,91155.58,706.0,2.572,23.01,NaN,8.9,NaN
112704,Y2023M03D18H19M55,NaN,None,91155.58,706.0,1.543,22.75,NaN,8.0,70.0


In [ ]:
    data3 = weatherData
    data3['cloud_layer_1_code_set_1' ] = data3['cloud_layer_1_code_set_1'].astype(int)
    data3['cloud_layer_1_code_set_1'] = data3['cloud_layer_1_code_set_1'].str.strip().str[-1]

In [ ]:
sun = requests.get('https://api.openweathermap.org/data/2.5/history/city?lat='+ str(airports[airport]['lat'])+ '&lon='+ str(airports[airport]['lon'])+'&appid=0bd19d55bfadd336edf90cb971181d59')

In [ ]:
sun=requests.get("https://history.openweathermap.org/data/2.5/history/city?lat=41.85&lon=-87&type=hour&start=1643720400&end=1643806800&units=metric&appid=0bd19d55bfadd336edf90cb971181d59")

In [ ]:
# sun = requests.get('https://api.openweathermap.org/data/2.5/history/city?q=Houston,TX&type=hour&start=1647658810&appid=0bd19d55bfadd336edf90cb971181d59')
# sun = requests.get("https://api.openweathermap.org/data/2.5/history/city?q=Houston&units=metric&appid=0bd19d55bfadd336edf90cb971181d59")
# sun = requests.get('http://history.openweathermap.org/data/3.0/history/locations/create?lat='+str(airports[airport]['lat'])+'&lon='+str(airports[airport]['lon'])+'&appid=0bd19d55bfadd336edf90cb971181d59')
# # sun = requests.get("https://api.solcast.com.au/world_radiation/estimated_actuals?latitude="+str(airports[airport]['lat'])+'&longitude='+str(airports[airport]['lon'])+'&hours=168')
# sun = requests.get("https://api.solcast.com.au/data/historic/radiation_and_weather?latitude="+str(airports[airport]['lat'])+'&longitude='+str(airports[airport]['lon'])+"&start_date=2022-03-18T14:00:00.000Z&duration=P300D&format=json&time_zone=utc")
# sun.json()


import requests

url = "https://api.solcast.com.au/data/historic/radiation_and_weather?latitude=-33.856784&longitude=151.215297&start_date=2022-10-25T14:45:00.000Z&duration=P30D&format=json&time_zone=utc"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)
response.text


''

In [ ]:
sun.json()
# weather2 = pd.DataFrame(sun.json()['STATION'][0]["OBSERVATIONS"])
# weather2

sun.json()


{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

In [ ]:
!pip install -U airportsdata


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00


In [ ]:
import airportsdata
airports = airportsdata.load()  # key is the ICAO identifier (the default)
print(airports[airport]['lat'])


33.6636666666667
